In [2]:
import pandas as pd
import gurobipy as gb
from gurobipy import GRB

# Load datasets
welders_data = pd.read_csv("/Users/huiyisang/Desktop/welders_data.csv")
welders_speed_data = pd.read_csv("/Users/huiyisang/Desktop/welders_speed_data.csv")

# Merge datasets on Welder_ID if necessary
if "Welder_ID" in welders_speed_data.columns:
    welders_data = welders_data.merge(welders_speed_data, on="Welder_ID", how="left")

# Extract relevant columns
welders_data = welders_data[["Welder_ID", "Safety_Rating", "Speed_Rating", 
                             "SMAW_Proficient", "GMAW_Proficient", 
                             "FCAW_Proficient", "GTAW_Proficient", "Experience_10_Years"]]

num_welders = len(welders_data)
num_welders_to_hire = 16

# Create Gurobi model
model = gb.Model("Welder Selection with Interaction Effects")

# Decision variables
x = model.addVars(num_welders, vtype=GRB.BINARY, name="WelderSelection")

# Objective function: Base speed + interaction effects
model.setObjective(
    gb.quicksum(welders_data.iloc[i]["Speed_Rating"] * x[i] for i in range(num_welders)) + 
    gb.quicksum(welders_speed_data.iloc[i, j] * x[i] * x[j] for i in range(num_welders) for j in range(num_welders) if i != j),
    GRB.MAXIMIZE
)

# Constraints
model.addConstr(gb.quicksum(x[i] for i in range(num_welders)) == num_welders_to_hire, "TotalWelders")

# Solve the nonlinear optimization problem
model.optimize()

# Display the new optimal objective function value
optimal_nonlinear_value = model.objVal
print(f"Optimal Objective Function Value (Nonlinear): {optimal_nonlinear_value}")


Set parameter Username
Set parameter LicenseID to value 2610034
Academic license - for non-commercial use only - expires 2026-01-14


TypeError: can't multiply sequence by non-int of type 'Var'